<h1>BLAST Algorithm<h1>

<h4>Packages required by this implementation: </h4><br>
    1. Biopython - allows reading FASTA format files<br>
    2. TQDM - a progress bar implementation<br>
    3. Pandas - data analysis and storage library that provides DataFrames as a storage structure<br>
    4. Pickle - storing and restoring different variables from and back into python  

In [1]:
from Bio import SeqIO
from tqdm import tqdm
import pandas as pd
import pickle

<h4>Functions<h4>
    <h5>getSeqDB(filename, ftype)</h5>
    Parses through all sequences in a given file and returns a Database in list form.

In [2]:
def getSeqDB(filename, ftype="fasta"):
    geneDB = []
    with open(filename, 'r') as allseqs:
        geneDB = list(SeqIO.parse(allseqs, ftype))
    return geneDB

<h5>getKmers(kemr_size:k, filename)</h5> 
Checks if there is a file for k kmer size availible and loads it. Otherwise creates a k size file.

In [3]:
def getKmers(k, filename):
    try:
        return loadKmers(k, filename)
    except:
        kmers = createKmers(k, getSeqDB(filename))
        saveKmers(kmers, str(k), filename)
        return kmers

<h5>createKmers(kmer_size:k, sequence_list:geneDB)</h5>
    Given a list of sequences, generates the kmers for each and saves it into a dictionary.

In [4]:
def createKmers(k, geneDB, kmers={}):
    for j in tqdm(range(len(geneDB)), desc="Creating Kmers DB"):
        seqrec = geneDB[j]
        seq = seqrec.seq
        for i in range(0, len(seq)-k+1):
            ind = str(seq[i:i+k])
            try: 
                kmers[ind] = kmers[ind] + [(seqrec.id, i)]
            except:
                kmers[ind] = [(seqrec.id , i)]
    return kmers

<h5>saveKmers(kmers, kmer_size:k, filename)</h5>
    Given a dictionary of kmers, kmers, saveKmers saves the list in a file using the Pickle library.
<h5>loadKmers(kmer_size:k, filename)</h5>
    Opens and loads the k-th size file.

In [5]:
def saveKmers(kmers, k, filename):
    with open(str(k)+'_kmers_'+filename+'_kb.pickle', 'wb') as kfile:
        pickle.dump(kmers, kfile)
        
def loadKmers(k, filename):
    with open(str(k)+'_kmers_'+filename+'_kb.pickle', 'rb') as kfile:
        return pickle.load(kfile)

<h5>matchSeq(input_sequence:test, kmers, kmer_size:k)</h5>
Creates a dataframe with each Kmer of the input sequence and matches it to all complete Kmer matches in the database

In [6]:
def matchSeq(test, kmers, k):
    row = {'Kmer': '_', 'SeqID':'None', 'Kmer Ind':-1, 'Seq Ind':-1}
    seqDB = []
    
    for i in tqdm(range(0, len(test)-k+1), desc="Matching Kmers"):
        ind = test[i:i+k]
        try:
            row['Kmer'] = ind
            row['Kmer Ind'] = i
            for seq in kmers[ind]:
                row['SeqID'] = seq[0]
                row['Seq Ind'] = seq[1]
                seqDB = seqDB+[dict(row)]
        except:
            pass
    return pd.DataFrame(seqDB)

<h5>combineMatches(input_match:seqDB)</h5>
Takes the matched DataFrame and yields the continous sequences from each queried sequence and database sequence in a subsequence object<br><br>

<p>Splits the matched DataFrame by the Sequence IDs<br>
    Reads every row within each splitted DataFrame<br>
    Checks for continuity by comparing each Index with previous index and expecting a difference of 1 for continuity, 0 for different match, and any other number for no more continuity.
    </p>

In [7]:
def combineMatches(seqDB):
    subseq = []
    seqDB = seqDB.sort_values(by='Kmer Ind')
    seqIDs = seqDB["SeqID"].unique()
    for k in tqdm(range(len(seqIDs)), desc="Combining Results"):
        s = seqIDs[k]
        tempDB = seqDB[seqDB["SeqID"]==s]
        s_seq = e_seq = []
        temp = []

        for j in range(0, len(tempDB)): 
            row = tempDB.iloc[j]
            rem = []
            toAdd = True

            for i in range(0, len(e_seq)):
                if row['Kmer Ind'] - e_seq[i][0] > 1:
                    rem = rem + [i]
                elif row['Kmer Ind'] == e_seq[i][0]:
                    if row['Seq Ind'] == e_seq[i][1]:
                        toAdd = False
                elif row['Kmer Ind'] - e_seq[i][0] == 1:
                    if row['Seq Ind'] - e_seq[i][1] == 1:
                        e_seq[i] = (row['Kmer Ind'], row['Seq Ind'])
                        toAdd = False
                    elif row['Seq Ind'] == e_seq[i][1]:
                        toAdd = False
                else:
                    print("HELP: ERROR")
            if toAdd:
                e_seq = e_seq + [(row['Kmer Ind'], row['Seq Ind'])]
                s_seq = s_seq + [(row['Kmer Ind'], row['Seq Ind'])]

            for i in range(0, len(rem)):
                temp = temp+[((s_seq[rem[i]-i][0], e_seq[rem[i]-i][0]), (s_seq[rem[i]-i][1], e_seq[rem[i]-i][1]))]
                del s_seq[rem[i]-i]
                del e_seq[rem[i]-i]

        for sS, eS in zip(s_seq, e_seq):
            temp = temp + [((sS[0], eS[0]), (sS[1], eS[1]))]
        subseq = subseq + [(temp, s)]
    return subseq

<h5>rankMatches(subsequence_object:subseq, kmer_size:k)</h5>
Converts the subsequence object into a DataFrame for better readability and conformity to output structure
<br><h6>NOTE: doesn't really rank the matches, just converts it into a format that can be ranked

In [8]:
def rankMatches(subseq, k):
    row = {'matching #': -1, 'inp start':-1, 'inp end':-1, 'inp length':-1, 'Seq ID':'None', 'seq start':-1, 'seq end':-1, 'seq length':-1}
    matchDF = []
    for i in tqdm(range(len(subseq)), desc="Converting to DataFrame"):
        v = subseq[i]
        row['Seq ID'] = v[1]
        for s in v[0]:
            row['matching #'] = s[0][1]-s[0][0]+1
            row['inp start'] = s[0][0]
            row['inp end'] = s[0][1]+k
            row['seq start'] = s[1][0]
            row['seq end'] = s[1][1]+k
            row['inp length'] = row['inp end'] - row['inp start']
            row['seq length'] = row['seq end'] - row['seq start']
            matchDF = matchDF + [dict(row)]

    return pd.DataFrame(matchDF)[['Seq ID', 'matching #', 'inp start', 'inp end', 'inp length', 'seq start', 'seq end', 'seq length']].sort_values(by='matching #', ascending=False).reset_index(drop=True)

<h4>input functions</h4>
Takes different inputs from the user to run the whole program

In [9]:
def u_input():
    print("1. Create Sequence Database")
    print("2. Append Sequence Database")
    print("3. Compare Sequences")
    print("4. Quit")
    
    c1 = int(input(prompt="Choose: "))      

    if c1==1:
        createDB()
    elif c1==2:
        appendDB()
    elif c1==3:
        comp = compSeq()
        return True, comp
    else:
        return False, False
    return True, False
    
def createDB():
    fname = str(input("Enter filename: "))
    k = int(input("Number of Kmers: "))
    getKmers(k, fname)

def appendDB():
    fname = str(input("Enter file to append to: "))
    filename = str(input("Enter file with sequences to: "))
    k = int(input("Number of Kmers: "))
    saveKmers(createKmers(filename, k, loadKmers(k, fname)), str(k), fname)

    
def compSeq():
    print("1. Default Comparison")
    print("2. Custom Comparison")
    
    c1 = int(input("Choose: "))
    if c1 == 1:
        k = 7
        filename = "small.fasta"
    else: 
        k = input("Size of Kmer: ")
        filename = input("Filename containing sequences: ")
    
    fname = str(input("Enter file to read Sequences from: "))
    seq = [x.seq for x in getSeqDB(fname)]
    print("Kmer DB...")
    kmers = getKmers(k, filename) 
    
    ret = []
    for i in range(len(seq)):
        s = seq[i]
        print("Matching...")
        seqDB = matchSeq(s, kmers, k)
        print("Combining...")
        subseq = combineMatches(seqDB)
        print("Displaying...")
        seqRank = rankMatches(subseq, k)
        seqRank['Query ID'] = pd.Series([i for _ in range(len(seqRank))])
        if (ret!=[]):
            ret = (pd.concat([ret[0], seqRank], ignore_index=True), pd.concat([ret[1], seqDB], ignore_index=True))
        else:
            ret = (seqRank, seqDB)
    return ret

def main():
    loop = True
    while(loop):
        loop, ans = u_input()
        if (type(ans) is not bool):
            return ans
    return False

In [10]:
seqList = main()
if (type(seqList) is not bool):
    print(seqList[0])

1. Create Sequence Database
2. Append Sequence Database
3. Compare Sequences
4. Quit


Choose:  3


1. Default Comparison
2. Custom Comparison


Choose:  1
Enter file to read Sequences from:  input_seq.fasta


Kmer DB...
Matching...


...

Combining...


Combining Results: 100%|█████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 22.24it/s]


Displaying...


Converting to DataFrame: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


                   Seq ID  matching #  inp start  inp end  inp length  \
0    sp|Q6GZX2|003R_FRG3G         432          0      438         438   
1   sp|Q8GYY0|1A112_ARATH           2        273      281           8   
2    sp|A1AN64|ACCD_PELPD           1         12       19           7   
3    sp|Q6GZV6|019R_FRG3G         845          0      851         851   
4    sp|Q6GZV6|019R_FRG3G          13         65       84          19   
5    sp|Q6GZV6|019R_FRG3G          13        135      154          19   
6    sp|Q6GZV6|019R_FRG3G          10         12       28          16   
7    sp|Q6GZV6|019R_FRG3G          10        145      161          16   
8    sp|Q6GZV6|019R_FRG3G           9         22       37          15   
9    sp|Q6GZV6|019R_FRG3G           9         85      100          15   
10   sp|Q6GZV6|019R_FRG3G           5        205      216          11   
11   sp|Q6GZV6|019R_FRG3G           5        371      382          11   
12   sp|Q6GZV6|019R_FRG3G           5        358   

In [ ]:
#Extra Function that ranks the sequences based on  either the # of matched Kmers or the frequency (defined as the sum of distance each Kmer Index is away from Sequence Index divided by the count of Kmers)

def rankSeq(seqDB, t, rCount=True):
    seqDB["Distance"] = seqDB.apply(lambda row: abs(row["Kmer Ind"]-row["Seq Ind"]), axis=1)
    seqRank = seqDB[['SeqID', 'Kmer']].groupby(by="SeqID").count().join(seqDB[['SeqID', 'Distance']].groupby(by="SeqID").sum())
    seqRank["Freq"] = seqRank.apply(lambda row: row["Distance"]/row["Kmer"], axis=1)
    seqRank = seqRank[seqRank['Kmer'].map(int)>t]
    if (rCount):
        return seqRank.sort_values(by="Kmer", ascending=False), seqDB
    else:
        return seqRank.sort_values(by="Freq").iloc[0:10], seqDB